# Using sktime models

This is an example for using sktime based models with the `timecopilot` library.

## imports

In [1]:
import nest_asyncio
nest_asyncio.apply()

import timecopilot
import sktime as skt
import pandas as pd

## Setup the sktime model

Sktime models can be passed in the `forecasters` argument when initializing the TimeCopilot agent where they will be wrapped in an adapter with an alias based on the type name. 

If multiple sktime forecasters of the same type are passed, each model after the first will have be wrapped in an adapter with an alias that has `'_n'` appended to it with `n` being incremented by 1 for each additional occurrence of the same model type. 

For example, if you pass two `TrendForecaster` sktime models, the first one will have an alias of `'sktime.TrendForecaster'` and the second one will have an alias of `'sktime.TrendForecaster_2'`.

If you would rather specify the alias yourself, you will need to adapt the model manually with `SKTimeAdapter`.

In [2]:
from sktime.forecasting.trend import TrendForecaster

trend_forecaster = TrendForecaster()

### Manually adapt sktime model

If you would rather decide on the alias yourself, you will need to manually adapt the model with `SKTimeAdapter`.

The `model` argument should be an sktime `Forecaster` model. The `alias` argument should be a string that uniquely identifies the model.

After adapting the model you would pass it in the `forecasters` argument when initializing the TimeCopilot agent.

If you add multiple manually adapted sktime models of the same type without specifying aliases, TimeCopilot may not be able to properly call all of them.

In [ ]:
from sktime.forecasting.trend import TrendForecaster
from timecopilot.models.adapters.sktime import SKTimeAdapter

trend_forecaster = TrendForecaster()

manually_adapted_model = SKTimeAdapter(
    model=trend_forecaster,
    alias="TrendForecaster",
)

tc = timecopilot.TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=[
        manually_adapted_model
    ]
)

## Create a TimeCopilot instance with your sktime model

You will need to specify the forecasters you're using when using sktime models. 

In [3]:
tc = timecopilot.TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=[
        trend_forecaster,
    ],
)

### Extending default model list with an sktime model

if you want to use the default list with the addition of your sktime model you could make a copy of the default list and append your model to it:

In [ ]:
model_list = timecopilot.agent.DEFAULT_MODELS.copy()
model_list.append(trend_forecaster)

tc = timecopilot.TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=model_list
)

## Forecasting 
Once that setup is complete, you can use TimeCopilot with your adapted sktime model the same way you'd normally use TimeCopilot

In [4]:
df = pd.read_csv("https://timecopilot.s3.amazonaws.com/public/data/air_passengers.csv")

In [5]:
result = tc.forecast(
    df=df,
)

1it [00:00,  4.52it/s]
1it [00:00, 220.94it/s]
11it [00:00, 83.97it/s]


In [6]:
print(result.output.tsfeatures_analysis)

The 'AirPassengers' data exhibits a strong trend with a stability of 0.933, indicating a consistent pattern over time. It also has a very strong seasonal component (seasonal_strength 0.982) with a seasonal period of 12 months, typical for annual data. The high x_acf1 (0.948) indicates strong autocorrelation, implying that past values are highly predictive of future values, suggesting that a time series model with a trend and seasonal structure would be appropriate. The entropy of the series is relatively low (0.429), indicating a high level of predictability, which substantiates the use of deterministic trend models. These features warrant the use of models that incorporate seasonality and trends, such as Holt's linear trend method or even seasonal naive models.


In [7]:
result.fcst_df

,unique_id,ds,sktime.TrendForecaster
0,AirPassengers,1961-01-01,473.023018
1,AirPassengers,1961-02-01,475.729097
2,AirPassengers,1961-03-01,478.173296
3,AirPassengers,1961-04-01,480.879374
4,AirPassengers,1961-05-01,483.498159
5,AirPassengers,1961-06-01,486.204237
6,AirPassengers,1961-07-01,488.823023
7,AirPassengers,1961-08-01,491.529101
8,AirPassengers,1961-09-01,494.235179
9,AirPassengers,1961-10-01,496.853964
